# Use the Entso-e API to download the data programatically

Download the "day-ahead" data for Germany for: 
- Load forecast 
- Load realised
- Generation solar 
- Generation wind (onshore and offshore)

All units are in MW (Megawatts). 

"Day-ahead" data provides values for each quarter of an hour, i.e. 96 values per day.

In [1]:
import requests
import pandas as pd
import datetime
from bs4 import BeautifulSoup as bs

# Import dictionaries, security_token and day_ahead loading script

from Name_convention_dictionaries import DocumentTypeDict
from Name_convention_dictionaries import ProcessTypeDict
from Name_convention_dictionaries import AreaDict
from Name_convention_dictionaries import PsrTypeDict
from security_token import security_token
from Load_data_entsoe_day import Load_data_entsoe

# Reverse dictionaries to increase readibility

zone_dict = {v:k for k,v in AreaDict.items()}
document_type_dict = {v:k for k,v in DocumentTypeDict.items()}
process_type_dict = {v:k for k,v in ProcessTypeDict.items()}
generation_type_dict = {v:k for k,v in PsrTypeDict.items()}

In [2]:
# support functions
def str_to_datetime(date_str):
    return datetime.datetime.strptime(date_str, '%Y%m%d%H%M')

def datetime_to_str(date):
    return datetime.datetime.strftime(date, "%Y%m%d%H%M")

## Specify region (country name or bidding zone according to Entso-e) and time horizon
Check Name_convention_dictionaries.py if unsure how the country or bidding zone is named by Entso-e.

In [3]:
region = 'Germany'

# The Entso-e API only allows to download 380 days per call. So create a loop to download all data 
# Time-format: Year, Month, Day, Hour, Minute

num_of_years_to_download = 7
most_recent_date = '202010252345'
list_for_loop = [365]*num_of_years_to_download

## Download Load Data

In [4]:
# Define furhter parameter to download the data
process_type = ['Day ahead', 'Realised'] # saved as a list to download forecast and realised respectively
document_type = 'System total load'

In [5]:
last_date_to_download = most_recent_date

for year in list_for_loop:
    begin_time = datetime_to_str(str_to_datetime(last_date_to_download) - datetime.timedelta(days=year))
    end_time = last_date_to_download

    # loop over forecast and realised load
    for process in process_type:
        
        # generate url for API call

        url = "https://transparency.entsoe.eu/api?securityToken="+security_token\
              +'&documentType='+document_type_dict[document_type]\
              +'&processType='+process_type_dict[process]\
              +'&outBiddingZone_Domain='+zone_dict[region]\
              +'&periodStart='+begin_time\
              +'&periodEnd='+end_time

        # load and save data
    
        try:
            Load_data_entsoe(url)
            last_date_to_download = begin_time
        except:
            print('\n No more data available \n')

You downloaded data for Zone "10Y1001A1001A83F" which is "Germany"
--------------------------------------------------------------------------------------------------
The downloaded time period is from 2019-10-26 22:00:00 until 2020-10-26 23:00:00 
--------------------------------------------------------------------------------------------------
The loaded document type is "A65" which corresponds to: "System total load"
The loaded process type is "A01" which corresponds to: "Day ahead" ,other examples: day Ahead, week ahead
######################################################
Sucessfully saved to:
Germany__&docType=A65__&rpocessType=A01from2019-10-26_22:00to2020-10-26_23:00.csv
######################################################



You downloaded data for Zone "10Y1001A1001A83F" which is "Germany"
--------------------------------------------------------------------------------------------------
The downloaded time period is from 2019-10-26 23:45:00 until 2020-10-25 23:45:00 
------

You downloaded data for Zone "10Y1001A1001A83F" which is "Germany"
--------------------------------------------------------------------------------------------------
The downloaded time period is from 2014-12-31 23:00:00 until 2015-10-27 23:45:00 
--------------------------------------------------------------------------------------------------
The loaded document type is "A65" which corresponds to: "System total load"
The loaded process type is "A16" which corresponds to: "Realised" ,other examples: day Ahead, week ahead
######################################################
Sucessfully saved to:
Germany__&docType=A65__&rpocessType=A16from2014-12-31_23:00to2015-10-27_23:45.csv
######################################################



<Reason>
<code>999</code>
<text>No matching data found for Data item Day-ahead Total Load Forecast [6.1.B] (10Y1001A1001A83F) and interval 2013-10-27T23:45:00.000Z/2014-10-27T23:45:00.000Z.</text>
</Reason>

 No more data available 

<Reason>
<code>999</c

## Download generation data

This is a little bit more complex than the load data. Therefore it is necessary to first download the generation actuals (realised) and then the forecast. 

Download the actuals/realised values for generation:

In [6]:
# Define furhter parameter to download the data
process_type = 'Realised' 
document_type = 'Actual generation per type'
generation_type = ['Solar', 'Wind Offshore', 'Wind Onshore']

In [7]:
last_date_to_download = most_recent_date

for year in list_for_loop:
    begin_time = datetime_to_str(str_to_datetime(last_date_to_download) - datetime.timedelta(days=year))
    end_time = last_date_to_download

    # loop over the generation types
    for generation in generation_type:
    
        # generate url for API call

        url = "https://transparency.entsoe.eu/api?securityToken="+security_token\
              +'&documentType='+document_type_dict[document_type]\
              +'&processType='+process_type_dict[process_type]\
              +'&psrType='+generation_type_dict[generation]\
              +'&in_Domain='+zone_dict[region]\
              +'&periodStart='+begin_time\
              +'&periodEnd='+end_time

        # load and save data

        try:
            Load_data_entsoe(url)
            last_date_to_download = begin_time
        except:
            print('\n No more data available \n')

You downloaded data for Zone "10Y1001A1001A83F" which is "Germany"
--------------------------------------------------------------------------------------------------
The downloaded time period is from 2019-10-26 23:45:00 until 2020-10-25 23:45:00 
--------------------------------------------------------------------------------------------------
The loaded document type is "A75" which corresponds to: "Actual generation per type"
The loaded process type is "A16" which corresponds to: "Realised" ,other examples: day Ahead, week ahead
The loaded prsrtype is "B16" which corresponds to: "Solar" 
######################################################
Sucessfully saved to:
Germany__&docType=A75__&rpocessType=A16__&psrType=B16from2019-10-26_23:45to2020-10-25_23:45.csv
######################################################



You downloaded data for Zone "10Y1001A1001A83F" which is "Germany"
--------------------------------------------------------------------------------------------------
The do

You downloaded data for Zone "10Y1001A1001A83F" which is "Germany"
--------------------------------------------------------------------------------------------------
The downloaded time period is from 2016-10-26 23:45:00 until 2017-10-26 23:45:00 
--------------------------------------------------------------------------------------------------
The loaded document type is "A75" which corresponds to: "Actual generation per type"
The loaded process type is "A16" which corresponds to: "Realised" ,other examples: day Ahead, week ahead
The loaded prsrtype is "B18" which corresponds to: "Wind Offshore" 
######################################################
Sucessfully saved to:
Germany__&docType=A75__&rpocessType=A16__&psrType=B18from2016-10-26_23:45to2017-10-26_23:45.csv
######################################################



You downloaded data for Zone "10Y1001A1001A83F" which is "Germany"
-------------------------------------------------------------------------------------------------

Download the forecasted generation for solar and wind

In [8]:
# Define furhter parameter to download the data
process_type = 'Day ahead' 
document_type = 'Wind and solar forecast'
generation_type = ['Solar', 'Wind Offshore', 'Wind Onshore']

In [9]:
last_date_to_download = most_recent_date

for year in list_for_loop:
    begin_time = datetime_to_str(str_to_datetime(last_date_to_download) - datetime.timedelta(days=year))
    end_time = last_date_to_download

    # loop over the generation types
    for generation in generation_type:
    
        # generate url for API call

        url = "https://transparency.entsoe.eu/api?securityToken="+security_token\
              +'&documentType='+document_type_dict[document_type]\
              +'&processType='+process_type_dict[process_type]\
              +'&psrType='+generation_type_dict[generation]\
              +'&in_Domain='+zone_dict[region]\
              +'&periodStart='+begin_time\
              +'&periodEnd='+end_time

        # load and save data

        try:
            Load_data_entsoe(url)
            last_date_to_download = begin_time
        except:
            print('\n No more data available \n')

You downloaded data for Zone "10Y1001A1001A83F" which is "Germany"
--------------------------------------------------------------------------------------------------
The downloaded time period is from 2019-10-26 22:00:00 until 2020-10-26 23:00:00 
--------------------------------------------------------------------------------------------------
The loaded document type is "A69" which corresponds to: "Wind and solar forecast"
The loaded process type is "A01" which corresponds to: "Day ahead" ,other examples: day Ahead, week ahead
The loaded prsrtype is "B16" which corresponds to: "Solar" 
######################################################
Sucessfully saved to:
Germany__&docType=A69__&rpocessType=A01__&psrType=B16from2019-10-26_22:00to2020-10-26_23:00.csv
######################################################



You downloaded data for Zone "10Y1001A1001A83F" which is "Germany"
--------------------------------------------------------------------------------------------------
The down

You downloaded data for Zone "10Y1001A1001A83F" which is "Germany"
--------------------------------------------------------------------------------------------------
The downloaded time period is from 2016-10-26 22:00:00 until 2017-10-27 22:00:00 
--------------------------------------------------------------------------------------------------
The loaded document type is "A69" which corresponds to: "Wind and solar forecast"
The loaded process type is "A01" which corresponds to: "Day ahead" ,other examples: day Ahead, week ahead
The loaded prsrtype is "B18" which corresponds to: "Wind Offshore" 
######################################################
Sucessfully saved to:
Germany__&docType=A69__&rpocessType=A01__&psrType=B18from2016-10-26_22:00to2017-10-27_22:00.csv
######################################################



You downloaded data for Zone "10Y1001A1001A83F" which is "Germany"
--------------------------------------------------------------------------------------------------
